# Easier unit testing through the creation of empty DataSets from schemas
We provide helper functions to generate (partially) empty DataSets from existing schemas. This can be helpful in certain situations, such as unit testing.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.Builder().config("spark.ui.showConsoleProgress", "false").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [2]:
from typedspark import Column, Schema, create_empty_dataset, create_partially_filled_dataset
from pyspark.sql.types import LongType, StringType

class Person(Schema):
    id: Column[LongType]
    name: Column[StringType]
    age: Column[LongType]

df_empty = create_empty_dataset(spark, Person)
df_empty.show()

+----+----+----+
|  id|name| age|
+----+----+----+
|null|null|null|
|null|null|null|
|null|null|null|
+----+----+----+



In [3]:
df_partially_filled = create_partially_filled_dataset(
    spark,
    Person,
    {
        Person.id: [1, 2, 3],
        Person.name: ["John", "Jane", "Jack"],
    }
)
df_partially_filled.show()

+---+----+----+
| id|name| age|
+---+----+----+
|  1|John|null|
|  2|Jane|null|
|  3|Jack|null|
+---+----+----+



## Complex datatypes

This trick also works for more complex data types.

In [4]:
from typedspark import ArrayType, MapType, StructType

class A(Schema):
    a: Column[StringType]
    b: Column[StringType]

class B(Schema):
    a: Column[ArrayType[StringType]]
    b: Column[MapType[StringType, StringType]]
    c: Column[StructType[A]]

df_a = create_partially_filled_dataset(
    spark, A, {
        A.a: ["a", "b", "c"]
    }
)

df_b = create_partially_filled_dataset(
    spark, B, {
        B.a: [["a"], ["b", "c"], ["d"]],
        B.b: [{"a": "1"}, {"b": "2", "c": "3"}, {"d": "4"}],
        B.c: df_a.collect(),
    }
)
df_b.show()

+------+----------------+---------+
|     a|               b|        c|
+------+----------------+---------+
|   [a]|        {a -> 1}|{a, null}|
|[b, c]|{b -> 2, c -> 3}|{b, null}|
|   [d]|        {d -> 4}|{c, null}|
+------+----------------+---------+



In [5]:
from datetime import date, datetime
from decimal import Decimal
from pyspark.sql.types import DateType, DecimalType, TimestampType

class C(Schema):
    a: Column[DateType]
    b: Column[TimestampType]
    c: Column[DecimalType]

create_partially_filled_dataset(
    spark, 
    C, 
    {
        C.a: [date(2020, 1, 1)],
        C.b: [datetime(2020, 1, 1, 10, 15)],
        C.c: [Decimal(32)],
    }
).show()

+----------+-------------------+---+
|         a|                  b|  c|
+----------+-------------------+---+
|2020-01-01|2020-01-01 10:15:00| 32|
+----------+-------------------+---+



## Row-wise definition of your DataSets

It is also possible to define your DataSets in a row-wise fashion. This is useful for cases where you have few columns, but many rows!

In [6]:
create_partially_filled_dataset(
    spark, 
    Person, 
    [
        {Person.name: "Alice", Person.age: 20},
        {Person.name: "Bob", Person.age: 30},
        {Person.name: "Charlie", Person.age: 40},
        {Person.name: "Dave", Person.age: 50},
        {Person.name: "Eve", Person.age: 60},
        {Person.name: "Frank", Person.age: 70},
        {Person.name: "Grace", Person.age: 80},
    ]
).show()

+----+-------+---+
|  id|   name|age|
+----+-------+---+
|null|  Alice| 20|
|null|    Bob| 30|
|null|Charlie| 40|
|null|   Dave| 50|
|null|    Eve| 60|
|null|  Frank| 70|
|null|  Grace| 80|
+----+-------+---+



This format also works for the complex data types:

In [7]:
from typedspark import create_structtype_row

create_partially_filled_dataset(
    spark,
    B,
    [
        {
            B.a: ["a"], 
            B.b: {"a": "1"}, 
            B.c: create_structtype_row(A, {A.a: "a", A.b: "b"}),
        },
        {
            B.a: ["b", "c"],
            B.b: {"b": "2", "c": "3"},
            B.c: create_structtype_row(A, {A.a: "b", A.b: "c"}),
        },
        {
            B.a: ["d"],
            B.b: {"d": "4"},
            B.c: create_structtype_row(A, {A.a: "d", A.b: "e"}),
        },
    ],
).show()

+------+----------------+------+
|     a|               b|     c|
+------+----------------+------+
|   [a]|        {a -> 1}|{a, b}|
|[b, c]|{b -> 2, c -> 3}|{b, c}|
|   [d]|        {d -> 4}|{d, e}|
+------+----------------+------+



## Example unit test

The following code snippet shows what a full unit test using typedspark can look like.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType, StringType
from typedspark import Column, DataSet, Schema, create_partially_filled_dataset, transform_to_schema
from chispa.dataframe_comparer import assert_df_equality


class Person(Schema):
    name: Column[StringType]
    age: Column[LongType]


def birthday(df: DataSet[Person]) -> DataSet[Person]:
    return transform_to_schema(df, Person, {Person.age: Person.age + 1})


def test_birthday(spark: SparkSession):
    df = create_partially_filled_dataset(
        spark, Person, {Person.name: ["Alice", "Bob"], Person.age: [20, 30]}
    )

    observed = birthday(df)
    expected = create_partially_filled_dataset(
        spark, Person, {Person.name: ["Alice", "Bob"], Person.age: [21, 31]}
    )

    assert_df_equality(observed, expected, ignore_row_order=True, ignore_nullable=True)
